In [6]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp -r "/content/drive/My Drive/Colab/mosquito/Final_Mosquito.csv" ./Final_Mosquito.csv

In [2]:
!pip install sentinelsat
!pip install geopandas
!pip install rasterio
!pip install pycrs

     |████████████████████████████████| 931kB 2.8MB/s 
     |████████████████████████████████| 10.4MB 16.8MB/s 
     |████████████████████████████████| 14.7MB 311kB/s 
     |████████████████████████████████| 18.1MB 237kB/s 
  Created wheel for pycrs: filename=PyCRS-1.0.2-cp36-none-any.whl size=32683 sha256=747bc70547782da539d2f056643ca15268b5a825de1aa6c41c18936d037f2d61
  Stored in directory: /root/.cache/pip/wheels/1a/c9/56/648f762bd41fdb03454be51b8a7069795925168833cb61f316
Successfully built pycrs


In [10]:
import random
print(random.uniform(38.3818, 38.8068))

random.uniform(-121.5565, -121.1321)

38.56557357550673


-121.44577135631312

In [0]:
import math
import json

from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt 
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import collections
import os
import shapely
from shapely.geometry import MultiPolygon, Polygon
import rasterio as rio
from rasterio.plot import show
import rasterio.mask
import fiona
import cv2

import rasterio
from rasterio.plot import show
from rasterio.plot import show_hist
from rasterio.mask import mask
from shapely.geometry import box
import geopandas as gpd
from fiona.crs import from_epsg
import pycrs
import dateutil
from datetime import datetime, timezone
import rasterio as rio
from rasterio.plot import show
from rasterio.mask import mask
import scipy.misc
from numpy import save
import pprint
import zipfile
import shutil
import sys

user = 'mad2755' ## change this!
password = 'Passsentinel02.' ## change this!
api = SentinelAPI(user, password, 'https://scihub.copernicus.eu/dhus')

def build_geojson(lat0,lon0,lat1,lon1,lat2,lon2,lat3,lon3):
    geojson = {
      "type": "FeatureCollection",
      "features": [
        {"type": "Feature",
          "properties": {},
           "geometry": {
            "type": "Polygon",
            "coordinates": [
              [
                [
                  lon0,
                  lat0
                ],
                [
                  lon1,
                  lat1
                ],
                [
                  lon2,
                  lat2
                ],
                [
                  lon3,
                  lat3
                ],
                [
                  lon3,
                  lat3
                ],
                [
                  lon0,
                  lat0
                ]
              ]
            ]
          }
        }
      ]
    }
    return geojson

def generate_geojson(lat,lon):
    R = 6378137
    
    #offsets in meters, 1km is the usual distance that a mosquito can travel
    dn = 500
    de = 500

    #Coordinate offsets in radians
    dLat = dn/R
    dLon = de/(R* math.cos(math.pi*lat/180))

    #OffsetPosition, decimal degrees
    lat0 = lat + dLat * 180/math.pi
    lon0 = lon + dLon * 180/math.pi 
    lat1 = lat + dLat * 180/math.pi
    lon1 = lon - dLon * 180/math.pi 
    lat2 = lat - dLat * 180/math.pi
    lon2 = lon - dLon * 180/math.pi 
    lat3 = lat - dLat * 180/math.pi
    lon3 = lon + dLon * 180/math.pi 
    
    return build_geojson(lat0,lon0,lat1,lon1,lat2,lon2,lat3,lon3), lon3, lat3, lon1, lat1

def process_geojson(geojson, date):
    footprint = geojson_to_wkt(geojson)

    startDate = str(2019)+'0101'
    endDate = str(int(2020))+'0101'
    #print(startDate)
    #print(endDate)
    products = api.query(footprint,
                         date = (startDate, endDate),
                         platformname = 'Sentinel-2',
                         processinglevel = 'Level-2A',
                         cloudcoverpercentage = (0, 10))
    #print(len(products))
    #check of products contain the requested polygon.
    final_areas = collections.OrderedDict()
    ft = shapely.wkt.loads(footprint)

    for key, value in products.items():
        P = shapely.wkt.loads(value['footprint'])
        if P.geom_type == 'MultiPolygon':
            for pol in P:
                if pol.contains(ft):
                    final_areas[key] = value

        elif P.geom_type == 'Polygon':
            if pol.contains(ft):
                final_areas[key] = value
    
    return final_areas

def get_closer_area_to_date(proposed_areas, date):
    
    #csv_date = dateutil.parser.parse(date)
    csv_date = datetime.now()
    csv_date = datetime(2019, csv_date.month, csv_date.day, tzinfo=timezone.utc)
    old_date = datetime(2010, 5, 17, tzinfo=timezone.utc)
    now_date = datetime.now(timezone.utc)
    current_min = abs(old_date - now_date)
    areas_by_date = collections.OrderedDict()

    current_date = datetime.now(timezone.utc)
    for key, value in proposed_areas.items():
        s_date = value['endposition']
        sentinel_date = datetime(s_date.year, s_date.month, s_date.day, tzinfo=timezone.utc)

        if abs(csv_date - sentinel_date) < current_min:

            current_min = abs(csv_date - sentinel_date)
            current_date = sentinel_date
            areas_by_date.clear()
            areas_by_date[key] = value
            
    #print("result date",current_date)
    # convert to Pandas DataFrame
    products_df = api.to_dataframe(areas_by_date)
    
    #sort and limit to first 5 sorted products
    #products_df_sorted = products_df.sort_values(['cloudcoverpercentage'], ascending=[True])
    #products_df_sorted = products_df_sorted.head(1)

    # download sorted and reduced products
    api.download_all(products_df.index)
    
    uuid = list(products_df.index)[0]
    product_title = list(products_df["title"])[0]
    return uuid, product_title

def extract_sentinel(uuid, product_title):
    
    file_name = str(product_title) +'.zip'
    if os.path.exists(file_name):
        with zipfile.ZipFile(file_name) as zf:
         zf.extractall()
        os.remove(file_name)
        
    path = str(product_title) + '.SAFE/GRANULE'
    files = os.listdir(path)
    # pathA = str(product_title) + '.SAFE/GRANULE/' + str(files[0])
    files2 = 'IMG_DATA'
    pathB = str(product_title) + '.SAFE/GRANULE/' + str(files[0]) +'/' + str(files2) +'/R10m'
    files3 = os.listdir(pathB)
    pathB = str(product_title) + '.SAFE/GRANULE/' + str(files[0]) +'/' + str(files2) +'/R20m'
    files4 = os.listdir(pathB)
    pathB = str(product_title) + '.SAFE/GRANULE/' + str(files[0]) +'/' + str(files2) +'/R60m'
    files5 = os.listdir(pathB)
    
    path_b1 = str(product_title) + '.SAFE/GRANULE/' + str(files[0]) +'/' + str(files2) +'/R60m/' +str(files5[0][0:23] +'B01_60m.jp2')
    path_b2 = str(product_title) + '.SAFE/GRANULE/' + str(files[0]) +'/' + str(files2) +'/R10m/' +str(files3[0][0:23] +'B02_10m.jp2')
    path_b3 = str(product_title) + '.SAFE/GRANULE/' + str(files[0]) +'/' + str(files2) +'/R10m/' +str(files3[0][0:23] +'B03_10m.jp2')
    path_b4 = str(product_title) + '.SAFE/GRANULE/' + str(files[0]) +'/' + str(files2) +'/R10m/' +str(files3[0][0:23] +'B04_10m.jp2')
    path_b5 = str(product_title) + '.SAFE/GRANULE/' + str(files[0]) +'/' + str(files2) +'/R20m/' +str(files4[0][0:23] +'B05_20m.jp2')
    path_b6 = str(product_title) + '.SAFE/GRANULE/' + str(files[0]) +'/' + str(files2) +'/R20m/' +str(files4[0][0:23] +'B06_20m.jp2')
    path_b7 = str(product_title) + '.SAFE/GRANULE/' + str(files[0]) +'/' + str(files2) +'/R20m/' +str(files4[0][0:23] +'B07_20m.jp2')
    path_b8 = str(product_title) + '.SAFE/GRANULE/' + str(files[0]) +'/' + str(files2) +'/R10m/' +str(files3[0][0:23] +'B08_10m.jp2')
    path_b8a = str(product_title) + '.SAFE/GRANULE/' + str(files[0]) +'/' + str(files2) +'/R20m/' +str(files3[0][0:23] +'B8A_20m.jp2')
    path_b9 = str(product_title) + '.SAFE/GRANULE/' + str(files[0]) +'/' + str(files2) +'/R60m/' +str(files5[0][0:23] +'B09_60m.jp2')
    path_b11 = str(product_title) + '.SAFE/GRANULE/' + str(files[0]) +'/' + str(files2) +'/R20m/' +str(files4[0][0:23] +'B11_20m.jp2')
    path_b12 = str(product_title) + '.SAFE/GRANULE/' + str(files[0]) +'/' + str(files2) +'/R20m/' +str(files4[0][0:23] +'B12_20m.jp2')

    b1 = rio.open(path_b1)
    b2 = rio.open(path_b2)
    b3 = rio.open(path_b3)
    b4 = rio.open(path_b4)
    b5 = rio.open(path_b5)
    b6 = rio.open(path_b6)
    b7 = rio.open(path_b7)
    b8 = rio.open(path_b8)
    b8a = rio.open(path_b8a)
    b9 = rio.open(path_b9)
    b11 = rio.open(path_b11)
    b12 = rio.open(path_b12)

    return [b1,b2,b3,b4,b5,b6,b7,b8,b8a,b9,b11,b12]

def crop_save(sentinel_array,minx, miny, maxx, maxy, id_num, product_title):
    
    bbox = box(minx, miny, maxx, maxy)
    geo = gpd.GeoDataFrame({'geometry': bbox}, index=[0], crs=from_epsg(4326))
    geo = geo.to_crs(crs=sentinel_array[0].crs.data)

    def getFeatures(gdf):
        import json
        return [json.loads(gdf.to_json())['features'][0]['geometry']]

    coords = getFeatures(geo)
    cropped = []

    for img in sentinel_array:
        out_img, out_transform = mask(img, shapes=coords, crop=True)
        out_img = np.moveaxis(out_img, 0, -1)
        cropped.append(out_img)
        #print(out_img.shape)

    #show(cropped[1])
    for i in range(len(cropped)):
        cropped[i] = cv2.resize(cropped[i], dsize=(104, 104))
        cropped[i] = np.expand_dims(cropped[i], axis=0)
        #print(cropped[i].shape)
    
    sentinelImg = np.concatenate((cropped[0],cropped[1],cropped[2],cropped[3],cropped[4],
                                  cropped[5],cropped[6],cropped[7],cropped[8],cropped[9],cropped[10],cropped[11]), axis=0)

    print(id_num, " - ",sentinelImg.shape)
    save('/content/drive/My Drive/Colab/mosquito/Sentinel/data_mosquito_final/data'+str(id_num)+'.npy', sentinelImg)
    
    file_name =  str(product_title) + '.SAFE'
    shutil.rmtree( file_name)
    os.system("rm -rf ~/.local/share/Trash/*")

def download_image(latitude,longitude, id_num, date):
    print('downloading', id_num)
    geojson, lon3, lat3, lon1, lat1  = generate_geojson(latitude,longitude)
    proposed_areas = process_geojson(geojson, date)
    uuid, product_title = get_closer_area_to_date(proposed_areas, date)
    sentinel_array = extract_sentinel(uuid, product_title)
    crop_save(sentinel_array, lon3, lat3, lon1, lat1, id_num, product_title)
    return geojson

from sklearn.utils import shuffle
df = pd.read_csv('Final_Mosquito.csv')

#for index, row in df.iterrows():
    #if not os.path.exists('/content/drive/My Drive/Colab/mosquito/Sentinel/data_mosquito_final/data'+str(row['OBJECTID'])+'.npy'):

for i in range(333,2000):
    download_image(random.uniform(38.3818, 38.8068), random.uniform(-121.5565, -121.1321), i,'TRAPCOLLECT')


downloading 333


Downloading: 100%|██████████| 1.16G/1.16G [00:38<00:00, 29.8MB/s]
MD5 checksumming: 100%|██████████| 1.16G/1.16G [00:02<00:00, 430MB/s]
/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


333  -  (12, 104, 104)
downloading 334


Downloading: 100%|██████████| 1.16G/1.16G [00:38<00:00, 29.9MB/s]
MD5 checksumming: 100%|██████████| 1.16G/1.16G [00:02<00:00, 407MB/s]
/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


334  -  (12, 104, 104)
downloading 335


...

502  -  (12, 104, 104)
downloading 503


Downloading: 100%|██████████| 1.16G/1.16G [00:39<00:00, 29.3MB/s]
MD5 checksumming: 100%|██████████| 1.16G/1.16G [00:02<00:00, 421MB/s]
/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


503  -  (12, 104, 104)
downloading 504


Downloading:  10%|█         | 121M/1.16G [00:04<00:35, 29.4MB/s]

Buffered data was truncated after reaching the output size limit.

In [0]:
!python "/content/drive/My Drive/Colab/mosquito/Sentinel/sentinel_numpy1.py"

downloading 1325
Downloading: 100% 1.17G/1.17G [00:36<00:00, 32.3MB/s]
MD5 checksumming: 100% 1.17G/1.17G [00:03<00:00, 384MB/s]
/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
1325  -  (12, 104, 104)
downloading 1326
Downloading: 100% 1.17G/1.17G [00:54<00:00, 21.6MB/s]
MD5 checksumming: 100% 1.17G/1.17G [00:02<00:00, 414MB/s]
/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_

In [0]:
import math
import json

from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt 
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import collections
import os
import shapely
from shapely.geometry import MultiPolygon, Polygon
import rasterio as rio
from rasterio.plot import show
import rasterio.mask
import fiona
import cv2

import rasterio
from rasterio.plot import show
from rasterio.plot import show_hist
from rasterio.mask import mask
from shapely.geometry import box
import geopandas as gpd
from fiona.crs import from_epsg
import pycrs
import dateutil
from datetime import datetime, timezone
import rasterio as rio
from rasterio.plot import show
from rasterio.mask import mask
import scipy.misc
from numpy import save
import pprint
import zipfile
import shutil
import sys


In [0]:
!pip install pycrs

In [0]:
user = 'mad2755' ## change this!
password = 'Passsentinel02.' ## change this!
api = SentinelAPI(user, password, 'https://scihub.copernicus.eu/dhus')

In [0]:
def build_geojson(lat0,lon0,lat1,lon1,lat2,lon2,lat3,lon3):
    geojson = {
      "type": "FeatureCollection",
      "features": [
        {"type": "Feature",
          "properties": {},
           "geometry": {
            "type": "Polygon",
            "coordinates": [
              [
                [
                  lon0,
                  lat0
                ],
                [
                  lon1,
                  lat1
                ],
                [
                  lon2,
                  lat2
                ],
                [
                  lon3,
                  lat3
                ],
                [
                  lon3,
                  lat3
                ],
                [
                  lon0,
                  lat0
                ]
              ]
            ]
          }
        }
      ]
    }
    return geojson

In [0]:
def generate_geojson(lat,lon):
    #Earth’s radius, sphere
    R = 6378137
    
    #offsets in meters, 1km is the usual distance that a mosquito can travel
    dn = 500
    de = 500

    #Coordinate offsets in radians
    dLat = dn/R
    dLon = de/(R* math.cos(math.pi*lat/180))

    #OffsetPosition, decimal degrees
    lat0 = lat + dLat * 180/math.pi
    lon0 = lon + dLon * 180/math.pi 
    lat1 = lat + dLat * 180/math.pi
    lon1 = lon - dLon * 180/math.pi 
    lat2 = lat - dLat * 180/math.pi
    lon2 = lon - dLon * 180/math.pi 
    lat3 = lat - dLat * 180/math.pi
    lon3 = lon + dLon * 180/math.pi 
    
    return build_geojson(lat0,lon0,lat1,lon1,lat2,lon2,lat3,lon3), lon3, lat3, lon1, lat1


In [0]:
def process_geojson(geojson, date):
    footprint = geojson_to_wkt(geojson)

    startDate = str(2019)+'0101'
    endDate = str(int(2020))+'0101'
    #print(startDate)
    #print(endDate)
    products = api.query(footprint,
                         date = (startDate, endDate),
                         platformname = 'Sentinel-2',
                         processinglevel = 'Level-2A',
                         cloudcoverpercentage = (0, 40))
    #print(len(products))
    #check of products contain the requested polygon.
    final_areas = collections.OrderedDict()
    ft = shapely.wkt.loads(footprint)

    for key, value in products.items():
        P = shapely.wkt.loads(value['footprint'])
        if P.geom_type == 'MultiPolygon':
            for pol in P:
                if pol.contains(ft):
                    final_areas[key] = value

        elif P.geom_type == 'Polygon':
            if pol.contains(ft):
                final_areas[key] = value
    
    return final_areas


In [0]:
def get_closer_area_to_date(proposed_areas, date):
    
    csv_date = dateutil.parser.parse(date)
    csv_date = datetime(2019, csv_date.month, csv_date.day, tzinfo=timezone.utc)
    old_date = datetime(2010, 5, 17, tzinfo=timezone.utc)
    now_date = datetime.now(timezone.utc)
    current_min = abs(old_date - now_date)
    areas_by_date = collections.OrderedDict()

    current_date = datetime.now(timezone.utc)
    for key, value in proposed_areas.items():
        s_date = value['endposition']
        sentinel_date = datetime(s_date.year, s_date.month, s_date.day, tzinfo=timezone.utc)

        if abs(csv_date - sentinel_date) < current_min:

            current_min = abs(csv_date - sentinel_date)
            current_date = sentinel_date
            areas_by_date.clear()
            areas_by_date[key] = value
            
    #print("result date",current_date)
    # convert to Pandas DataFrame
    products_df = api.to_dataframe(areas_by_date)
    
    #sort and limit to first 5 sorted products
    #products_df_sorted = products_df.sort_values(['cloudcoverpercentage'], ascending=[True])
    #products_df_sorted = products_df_sorted.head(1)

    # download sorted and reduced products
    api.download_all(products_df.index)
    
    uuid = list(products_df.index)[0]
    product_title = list(products_df["title"])[0]
    return uuid, product_title

In [0]:
def extract_sentinel(uuid, product_title):
    
    file_name = str(product_title) +'.zip'
    if os.path.exists(file_name):
        with zipfile.ZipFile(file_name) as zf:
         zf.extractall()
        os.remove(file_name)
        
    path = str(product_title) + '.SAFE/GRANULE'
    files = os.listdir(path)
    pathA = str(product_title) + '.SAFE/GRANULE/' + str(files[0])
    files2 = os.listdir(pathA)
    pathB = str(product_title) + '.SAFE/GRANULE/' + str(files[0]) +'/' + 'IMG_DATA' +'/R10m'
    files3 = os.listdir(pathB)
    pathB = str(product_title) + '.SAFE/GRANULE/' + str(files[0]) +'/' + 'IMG_DATA' +'/R20m'
    files4 = os.listdir(pathB)
    pathB = str(product_title) + '.SAFE/GRANULE/' + str(files[0]) +'/' + 'IMG_DATA' +'/R60m'
    files5 = os.listdir(pathB)
    
    path_b1 = str(product_title) + '.SAFE/GRANULE/' + str(files[0]) +'/' + 'IMG_DATA' +'/R60m/' +str(files5[0][0:23] +'B02_60m.jp2')
    path_b2 = str(product_title) + '.SAFE/GRANULE/' + str(files[0]) +'/' + 'IMG_DATA' +'/R10m/' +str(files3[0][0:23] +'B02_10m.jp2')
    path_b3 = str(product_title) + '.SAFE/GRANULE/' + str(files[0]) +'/' + 'IMG_DATA' +'/R10m/' +str(files3[0][0:23] +'B03_10m.jp2')
    path_b4 = str(product_title) + '.SAFE/GRANULE/' + str(files[0]) +'/' + 'IMG_DATA' +'/R10m/' +str(files3[0][0:23] +'B04_10m.jp2')
    path_b5 = str(product_title) + '.SAFE/GRANULE/' + str(files[0]) +'/' + 'IMG_DATA' +'/R20m/' +str(files4[0][0:23] +'B05_20m.jp2')
    path_b6 = str(product_title) + '.SAFE/GRANULE/' + str(files[0]) +'/' + 'IMG_DATA' +'/R20m/' +str(files4[0][0:23] +'B06_20m.jp2')
    path_b7 = str(product_title) + '.SAFE/GRANULE/' + str(files[0]) +'/' + 'IMG_DATA' +'/R20m/' +str(files4[0][0:23] +'B07_20m.jp2')
    path_b8 = str(product_title) + '.SAFE/GRANULE/' + str(files[0]) +'/' + 'IMG_DATA' +'/R10m/' +str(files3[0][0:23] +'B08_10m.jp2')
    path_b9 = str(product_title) + '.SAFE/GRANULE/' + str(files[0]) +'/' + 'IMG_DATA' +'/R60m/' +str(files5[0][0:23] +'B09_60m.jp2')
    path_b11 = str(product_title) + '.SAFE/GRANULE/' + str(files[0]) +'/' + 'IMG_DATA' +'/R20m/' +str(files4[0][0:23] +'B11_20m.jp2')
    path_b12 = str(product_title) + '.SAFE/GRANULE/' + str(files[0]) +'/' + 'IMG_DATA'+'/R20m/' +str(files4[0][0:23] +'B12_20m.jp2')

    b1 = rio.open(path_b1)
    b2 = rio.open(path_b2)
    b3 = rio.open(path_b3)
    b4 = rio.open(path_b4)
    b5 = rio.open(path_b5)
    b6 = rio.open(path_b6)
    b7 = rio.open(path_b7)
    b8 = rio.open(path_b8)
    b9 = rio.open(path_b9)
    b11 = rio.open(path_b11)
    b12 = rio.open(path_b12)

    return [b1,b2,b3,b4,b5,b6,b7,b8,b9,b11,b12]

In [0]:
def crop_save(sentinel_array,minx, miny, maxx, maxy, id_num, product_title):
    
    bbox = box(minx, miny, maxx, maxy)
    geo = gpd.GeoDataFrame({'geometry': bbox}, index=[0], crs=from_epsg(4326))
    geo = geo.to_crs(crs=sentinel_array[0].crs.data)

    def getFeatures(gdf):
        import json
        return [json.loads(gdf.to_json())['features'][0]['geometry']]

    coords = getFeatures(geo)
    cropped = []

    for img in sentinel_array:
        out_img, out_transform = mask(img, shapes=coords, crop=True)
        out_img = np.moveaxis(out_img, 0, -1)
        cropped.append(out_img)
        #print(out_img.shape)

    #show(cropped[1])
    for i in range(len(cropped)):
        cropped[i] = cv2.resize(cropped[i], dsize=(104, 104))
        cropped[i] = np.expand_dims(cropped[i], axis=0)
        #print(cropped[i].shape)
    
    sentinelImg = np.concatenate((cropped[0],cropped[1],cropped[2],cropped[3],cropped[4],
                                  cropped[5],cropped[6],cropped[7],cropped[8],cropped[9],cropped[10]), axis=0)

    print(id_num, " - ",sentinelImg.shape)
    save('drive/My Drive/Colab/mosquito/data_mosquito/data'+str(id_num)+'.npy', sentinelImg)
    
    file_name =  str(product_title) + '.SAFE'
    shutil.rmtree( file_name)
    os.system("rm -rf ~/.local/share/Trash/*")

In [0]:
def download_image(latitude,longitude, id_num, date):
    print('downloading', id_num)
    geojson, lon3, lat3, lon1, lat1  = generate_geojson(latitude,longitude)
    proposed_areas = process_geojson(geojson, date)
    uuid, product_title = get_closer_area_to_date(proposed_areas, date)
    sentinel_array = extract_sentinel(uuid, product_title)
    crop_save(sentinel_array, lon3, lat3, lon1, lat1, id_num, product_title)
    return geojson

downloading 1991


Downloading: 100%|██████████| 1.18G/1.18G [00:46<00:00, 25.2MB/s]
MD5 checksumming: 100%|██████████| 1.18G/1.18G [00:02<00:00, 417MB/s]
/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


1991  -  (11, 104, 104)
downloading 1992


Downloading: 100%|██████████| 1.18G/1.18G [00:42<00:00, 27.5MB/s]
MD5 checksumming: 100%|██████████| 1.18G/1.18G [00:02<00:00, 426MB/s]
/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


1992  -  (11, 104, 104)
downloading 1993


...

1896  -  (11, 104, 104)
downloading 1897


Downloading: 100%|██████████| 1.18G/1.18G [00:38<00:00, 30.2MB/s]
MD5 checksumming: 100%|██████████| 1.18G/1.18G [00:02<00:00, 422MB/s]
/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


1897  -  (11, 104, 104)
downloading 1898


Downloading:  66%|██████▌   | 772M/1.18G [00:25<00:13, 29.9MB/s]


KeyboardInterrupt: ignored

In [0]:
df = pd.read_csv("/content/drive/My Drive/Colab/mosquito/Final_Mosquito.csv")

for index, row in df.iterrows():
    
    if not os.path.exists('drive/My Drive/Colab/mosquito/data_mosquito/data'+str(row['OBJECTID'])+'.npy'):
        download_image(row['LATITUDE'], row['LONGITUDE'], row['OBJECTID'],row['TRAPCOLLECT'])


In [0]:
!pwd

/content


In [0]:
!cp -avr 'drive/My Drive/Colab/mosquito/data_mosquito' /content

'drive/My Drive/Colab/mosquito/data_mosquito' -> '/content/data_mosquito'
'drive/My Drive/Colab/mosquito/data_mosquito/data361.npy' -> '/content/data_mosquito/data361.npy'
'drive/My Drive/Colab/mosquito/data_mosquito/data363.npy' -> '/content/data_mosquito/data363.npy'
'drive/My Drive/Colab/mosquito/data_mosquito/data371.npy' -> '/content/data_mosquito/data371.npy'
'drive/My Drive/Colab/mosquito/data_mosquito/data368.npy' -> '/content/data_mosquito/data368.npy'
'drive/My Drive/Colab/mosquito/data_mosquito/data362.npy' -> '/content/data_mosquito/data362.npy'
'drive/My Drive/Colab/mosquito/data_mosquito/data365.npy' -> '/content/data_mosquito/data365.npy'
'drive/My Drive/Colab/mosquito/data_mosquito/data369.npy' -> '/content/data_mosquito/data369.npy'
'drive/My Drive/Colab/mosquito/data_mosquito/data366.npy' -> '/content/data_mosquito/data366.npy'
'drive/My Drive/Colab/mosquito/data_mosquito/data364.npy' -> '/content/data_mosquito/data364.npy'
'drive/My Drive/Colab/mosquito/data_mosquito